In [4]:
! pip install scikit-image

You should consider upgrading via the 'c:\users\dear user\anaconda\anaconda3\python.exe -m pip install --upgrade pip' command.



  Using cached PyWavelets-1.1.1-cp37-cp37m-win_amd64.whl (4.2 MB)


In [142]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import feature
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import glob
from skimage import img_as_ubyte
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import pickle
import random
from skimage.feature import hog
from skimage import data, exposure
from sklearn.decomposition import PCA 
from skimage.feature import greycomatrix, greycoprops

In [2]:
path = r'C:\folder\BOOT Camp\computer vision\04, Fourier transform\GENKI-R2009a\GENKI-R2009a\Subsets\GENKI-4K.txt\files\*.*'
SIZE = 120
images_list = [cv2.resize(cv2.imread(file, 0), (SIZE, SIZE)) for file in tqdm(glob.glob(path))] 

  0%|          | 0/4000 [00:00<?, ?it/s]

In [99]:
def lbf_feature(img, redius=1, sampling_pixels=8):
    
    # Compute LBP
    lbp = feature.local_binary_pattern(img, redius, sampling_pixels, method='uniform')
    
    return lbp.ravel()

In [100]:
def hog_feature(img):
    
    # Compute HOG
    hog_ =  hog(img, orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1))
    return hog_

In [146]:
images_list[0].shape

(120, 120)

In [165]:
math = [
    'contrast'
    'dissimilarity'
    'homogeneity'
    'ASM'
    'energy'
    'correlation'
    ]

def glcm_feature(img):
    glcm = greycomatrix(img, distances=[5], angles=[0], levels=256,
                            symmetric=True, normed=True)
    return greycoprops(glcm, 'dissimilarity')
#     



In [166]:
def iterator(functions, _iter):

    develope = _iter
    for func in tqdm(functions):
        develope = np.array(list(map(lambda element: eval(func), develope)))
        
    return develope

In [168]:
image_arrays = np.array(images_list)
lbf_hist = iterator(['cv2.GaussianBlur(element, (3, 3), cv2.BORDER_DEFAULT)',
                                   'cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8)).apply(element)',
                                   'lbf_feature(element)'], image_arrays) 



hog_f = iterator(['cv2.GaussianBlur(element, (3, 3), cv2.BORDER_DEFAULT)',
                                   'cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8)).apply(element)',
                                   'hog_feature(element)'], image_arrays) 

glcm_f = iterator(['cv2.GaussianBlur(element, (3, 3), cv2.BORDER_DEFAULT)',
                                   'cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8)).apply(element)',
                                   'glcm_feature(element)'], image_arrays)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [174]:
glcm_f.shape = (4000,1)

In [105]:
path = r'C:\folder\BOOT Camp\computer vision\04, Fourier transform\GENKI-R2009a\GENKI-R2009a\Subsets\GENKI-4K.txt\GENKI-4K_Labels.txt'

with open(path, 'r') as Labels_file:
    labels = list(map(lambda lin: lin[0], Labels_file.readlines()))


In [175]:
X_train =  np.concatenate([PCA(n_components=100).fit_transform(i) for i in [lbf_hist, hog_f]]+[glcm_f], axis=1)
y_train = np.array(labels)

data = [X_train, y_train]

In [176]:
with open('data.pickle', 'wb') as pickle_file:
    pickle.dump(data, pickle_file)

In [177]:
with open('data.pickle', 'rb') as pickle_file:
    data = pickle.load(pickle_file)

In [139]:
# GridSearchCV

param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
grid = GridSearchCV(SVC(), param_grid, verbose = 3)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   2.0s
[CV 2/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   2.2s
[CV 3/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   2.0s
[CV 4/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   2.2s
[CV 5/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   2.0s
[CV 1/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   1.6s
[CV 2/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   1.9s
[CV 3/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   1.6s
[CV 4/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   1.8s
[CV 5/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   1.9s
[CV 1/5] END ..................C=0.1, gamma=0.01, kernel=rbf; total time=   1.8s
[CV 2/5] END ..................C=0.1, gamma=0.0

[CV 2/5] END ....................C=1000, gamma=1, kernel=rbf; total time=   2.1s
[CV 3/5] END ....................C=1000, gamma=1, kernel=rbf; total time=   2.3s
[CV 4/5] END ....................C=1000, gamma=1, kernel=rbf; total time=   2.2s
[CV 5/5] END ....................C=1000, gamma=1, kernel=rbf; total time=   2.2s
[CV 1/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=   1.8s
[CV 2/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=   1.6s
[CV 3/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=   1.7s
[CV 4/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=   1.8s
[CV 5/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=   1.6s
[CV 1/5] END .................C=1000, gamma=0.01, kernel=rbf; total time=   1.7s
[CV 2/5] END .................C=1000, gamma=0.01, kernel=rbf; total time=   1.8s
[CV 3/5] END .................C=1000, gamma=0.01, kernel=rbf; total time=   1.6s
[CV 4/5] END ...............

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [178]:
print(grid.best_estimator_)

SVC(C=10, gamma=0.0001)


In [183]:
svm = SVC(C=10, gamma=0.0001)
svm = svm.fit(X_train, y_train)

In [179]:
Scores = cross_val_score(SVC(C=10, gamma=0.0001), X_train, y_train)

In [180]:
Scores.mean(), Scores.std()

(0.6412500000000001, 0.007416198487095695)

In [184]:
predictions = svm.predict(X_train)
print(classification_report(y_train, predictions))

              precision    recall  f1-score   support

           0       0.81      0.75      0.78      1838
           1       0.80      0.85      0.83      2162

    accuracy                           0.81      4000
   macro avg       0.81      0.80      0.80      4000
weighted avg       0.81      0.81      0.81      4000

